# **Interactive Visual Analytics with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


- Mark all launch sites on a map
 - Mark the success/failed launches for each site on the map
- Calculate the distances between a launch site to its proximities


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [5]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [10]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [11]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [37]:
latitude = launch_sites_df['Lat'].mean()
longitude = launch_sites_df['Long'].mean()
site_map = folium.Map(location=[latitude, longitude], zoom_start=4)

for index, row in launch_sites_df.iterrows():
    # Get the coordinates and name
    site_lat = row['Lat']
    site_long = row['Long']
    site_name = row['Launch Site']

    # Create a Marker object for the current site
    marker = folium.Marker(
        [site_lat, site_long],
        # Add a popup with the name of the launch site
        popup=site_name,
        # Use a distinctive icon for the launch site
        icon=folium.Icon(color='blue', icon='rocket')
    )

    # Add the marker to the map
    marker.add_to(site_map)

# Display the map with all markers
site_map

# Creating folium map

In [17]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [19]:
# Creating a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

# Creating a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Creating an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

# Creating and adding `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


In [20]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
nasa_coordinate = [launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, row in launch_sites_df.iterrows():
    # Get the coordinates and name
    coordinate = [row['Lat'], row['Long']]
    launch_site_name = row['Launch Site']

    # Create a Circle for the launch site (e.g., radius 1000m)
    # The example uses color='000000' (black) for the circle border
    folium.Circle(
        coordinate,
        radius=1000,
        color='#000000', # Black border color
        fill=True,
        # Fill color set to a light color like blue
        fill_color='#0066FF', 
        fill_opacity=0.4
    ).add_child(folium.Popup(launch_site_name)).add_to(site_map)
    
    # Create a Marker with a DivIcon for the launch site name
    # DivIcon is used to display the text label directly on the map
    folium.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_site_name,
        )
    ).add_to(site_map)

# Display the map (This will show the map in the output)
site_map

- Are all launch sites in proximity to the Equator line? 
- Are all launch sites in very close proximity to the coast?


In [28]:
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

# For each row in spacex_df data frame
# create a marker and add it to the marker_cluster
for index, record in spacex_df.iterrows():
    
    # Get the coordinates and the color
    coordinate = [record['Lat'], record['Long']]
    
    # Create a Marker object with its coordinate and customize its icon color
    # The color is taken from the 'marker_color' column (red for failed, green for success)
    marker = folium.Marker(
        coordinate,
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=f"Launch Site: {record['Launch Site']} | Class: {record['class']}"
    )
    
    # Add the marker to the MarkerCluster object
    marker_cluster.add_child(marker)

# Display the map with the colored clusters
site_map

# Enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.

In [29]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


# Create markers for all launch records.

In [30]:
marker_cluster = MarkerCluster()

# Creating a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [39]:
# Applying a function to check the value of `class` column

spacex_data = {
    'Launch Site': ['KSC LC-39A', 'KSC LC-39A', 'KSC LC-39A', 'CCAFS SLC-40', 'CCAFS SLC-40', 'CCAFS SLC-40', 'CCAFS SLC-40', 'CCAFS SLC-40', 'CCAFS SLC-40', 'CCAFS SLC-40'],
    'Lat': [28.573255, 28.573255, 28.573255, 28.563197, 28.563197, 28.563197, 28.563197, 28.563197, 28.563197, 28.563197],
    'Long': [-80.646895, -80.646895, -80.646895, -80.576820, -80.576820, -80.576820, -80.576820, -80.576820, -80.576820, -80.576820],
    'class': [1, 1, 1, 1, 1, 0, 0, 0, 1, 0] # Based on the provided table's success/failure
}

spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')
marker_cluster = MarkerCluster() 
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']] 

    # Create a Marker object
    marker = folium.Marker(
        coordinate, 
        icon=folium.Icon(color='white', icon_color=record['marker_color']), 
        popup=f"Launch Site: {record['Launch Site']} | Class: {record['class']}" 
    )
    # Add the marker to the MarkerCluster objec
    marker_cluster.add_child(marker) 


# Display the map
site_map

In [56]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)


for index, record in spacex_df.iterrows(): 
    marker_cluster.add_child(marker)

site_map

In [52]:
# Calculate the distances between a launch site to its proximities

from math import sin, cos, sqrt, atan2, radians 

def calculate_distance(lat1, lon1, lat2, lon2): 
    # approximate radius of earth in km
    R = 6373.0 

    lat1_rad = radians(lat1) 
    lon1_rad = radians(lon1) 
    lat2_rad = radians(lat2) 
    lon2_rad = radians(lon2) 

    dlon = lon2_rad - lon1_rad 
    dlat = lat2_rad - lat1_rad 

    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2 
    c = 2 * atan2(sqrt(a), sqrt(1 - a)) 
    
    distance_km = R * c 
    
    return distance_km * 1000

launch_site_coords = [launch_site_lat, launch_site_lon]

coastline_coords = [coastline_lat, coastline_lon]

Next, we need to explore and analyze the proximities of launch sites.


# add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [45]:
# Adding Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

distance_text = f"{distance_coastline/1000:.2f} KM"
distance_marker = folium.Marker(
    coastline_coords, 
    icon=DivIcon(
        icon_size=(20, 20), 
        icon_anchor=(0, 0), 
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance_text}</b></div>', 
    )
)
site_map.add_child(distance_marker)

coordinates = [launch_site_coords, coastline_coords]
lines = folium.PolyLine(locations=coordinates, weight=1) 
site_map.add_child(lines) 

site_map

In [46]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

 # Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [47]:
# find coordinate of the closet coastline

launch_site_lat = 28.563197
launch_site_lon = -80.576820
launch_site_coords = [launch_site_lat, launch_site_lon]

coastline_lat = 28.56367
coastline_lon = -80.57163
coastline_coords = [coastline_lat, coastline_lon]

distance_coastline_m = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline_km = distance_coastline_m / 1000

print(f"The distance to the closest coastline is: {distance_coastline_km:.2f} KM")

The distance to the closest coastline is: 0.00 KM


In [57]:
# Create and add a folium.Marker on your selected closest coastline point on the map

distance_marker = folium.Marker(
    coastline_coords,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance_coastline_km:.2f} KM</b></div>'
    )
)
site_map.add_child(distance_marker)


# The PolyLine needs a list of coordinates
coordinates = [launch_site_coords, coastline_coords]
lines = folium.PolyLine(locations=coordinates, weight=1)

# Add the PolyLine to the map
site_map.add_child(lines)

# Display the updated map (simulated)
# site_map

In [58]:
site_map.add_child(lines)

In [51]:
# Nearest Railway
railway_lat = 28.57211
railway_lon = -80.58525
railway_coords = [railway_lat, railway_lon]

# Nearest Highway
highway_lat = 28.56363
highway_lon = -80.80376
highway_coords = [highway_lat, highway_lon]

# Nearest City (e.g., Titusville)
city_lat = 28.61208
city_lon = -80.80802
city_coords = [city_lat, city_lon]

# --- Function to plot distance ---
def plot_distance_proximity(proximity_coords, launch_coords, proximity_name, map_obj):
    # Calculate distance
    lat1, lon1 = launch_coords
    lat2, lon2 = proximity_coords
    distance_m = calculate_distance(lat1, lon1, lat2, lon2)
    distance_km = distance_m / 1000

    print(f"Distance to {proximity_name}: {distance_km:.2f} KM")

    # Create a marker for the proximity point with distance displayed
    distance_text = f"{distance_km:.2f} KM"
    marker = folium.Marker(
        proximity_coords,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#0055AA;"><b>{proximity_name}<br>{distance_text}</b></div>'
        ),
        popup=f"{proximity_name} Distance: {distance_text}"
    )
    map_obj.add_child(marker)

    # Draw PolyLine connecting the launch site and the proximity point
    lines = folium.PolyLine(locations=[launch_coords, proximity_coords], weight=1, color='blue')
    map_obj.add_child(lines)

    return map_obj

# 1. Railway
site_map = plot_distance_proximity(railway_coords, launch_site_coords, "Railway", site_map)

# 2. Highway
site_map = plot_distance_proximity(highway_coords, launch_site_coords, "Highway", site_map)

# 3. City
site_map = plot_distance_proximity(city_coords, launch_site_coords, "City", site_map)

# Display the map (simulated)
site_map

Distance to Railway: 0.00 KM
Distance to Highway: 0.02 KM
Distance to City: 0.02 KM


*   Are launch sites in close proximity to railways? Yes
*   Are launch sites in close proximity to highways? 
*   Are launch sites in close proximity to coastline? Yes one of the coastline is found to be 0.87km away
*   Do launch sites keep certain distance away from cities? yes sice rocket parts can fall off it can be dangerous to keep it near to city on of distance from city is 2km

